# Call Centre Optimisation Simulation

## Key features
* Two agents, two topics
* Two queues, one for each agent. Once customer assigned to the queue they cannot move to another
* Both topic calls could go either queues
* Both agents can answer either topics, but one is better at one topic
to another (faster answer, higher satisfaction)
* Primary factor weight for decision making is Time (expected waiting time in the queue). [Optional] secondary factor is Quality (expected satisfaction rating)
* One of the topics questions takes longer to be answered

In [20]:
# Global environments
import numpy as np
from math import *
import simpy
import random

## Simpy environment
env = simpy.Environment()

## Simulation time
SIM_TIME = 1000

## Customer arrivals (Poisson distribution)
### Consider open ended function
LAMBDA_SIMPLE = 3 # Simple topic, 3 calls per time unit
LAMBDA_COMPLEX = 1 # Complex topic, 1 calls per time unit

## Service rate of the agents (Exponential distribution)
## Topic 1 (simple) rate will always be higher than Topic 2 (complex) rate
## Agent 1 is better at Topic 1, Agent 2 is better at Topic 2
### Consider open ended function
MU_1_SIMPLE = 1/2 # Agent 1 Topic 1
MU_2_SIMPLE = 1/3 # Agent 2 Topic 1
MU_2_COMPLEX = 1/4 # Agent 2 Topic 2
MU_1_COMPLEX = 1/5 # Agent 1 Topic 2

## Waiting time tracking
waiting_times = []


In [21]:
# Call Centre operation

class CallCentre:
    def __init__(self, env):
        self.env = env
        # Agents
        self.agent1 = simpy.Resource(env, capacity=1)
        self.agent2 = simpy.Resource(env, capacity=1)
        # Queues
        self.queue1 = simpy.Store(env)
        self.queue2 = simpy.Store(env)
        
        # Start agent processes
        self.env.process(self.agent_service(self.agent1, self.queue1, MU_1_SIMPLE, MU_1_COMPLEX))
        self.env.process(self.agent_service(self.agent2, self.queue2, MU_2_SIMPLE, MU_2_COMPLEX))

    def call_generator(self, topic, arrival_rate):
        while True:
            # Interarrival time
            interarrival = random.expovariate(arrival_rate)
            yield self.env.timeout(interarrival)
            arrival_time = self.env.now
            self.assign_queue(topic, arrival_time)
            
    def expected_waiting_time(self, queue, mu_simple, mu_complex):
        q_len = len(queue.items)
        if q_len == 0:
            return 0
        # Estimate service time based on topic distribution
        simple_count = sum([1 for i in queue.items if i[1] == 'simple'])
        complex_count = q_len - simple_count
        p1 = simple_count/q_len if q_len > 0 else 0
        p2 = complex_count/q_len if q_len > 0 else 0
        expected_service_time = p1/mu_simple + p2/mu_complex
        return (q_len + 1) / 2 * expected_service_time # M/M/1 queue
    
    def assign_queue(self, topic, arrival_time):
        ew1 = self.expected_waiting_time(self.queue1, MU_1_SIMPLE, MU_1_COMPLEX)
        ew2 = self.expected_waiting_time(self.queue2, MU_2_SIMPLE, MU_2_COMPLEX)
        
        if ew1 < ew2:
            yield self.queue1.put((arrival_time, topic))
        else:
            yield self.queue2.put((arrival_time, topic))
        
    
    def agent_service(self, agent, queue, mu_simple, mu_complex):
        while True:
            with agent.request() as req:
                yield req
                # if len(queue.items) == 0:
                #     return
                
                arrival_time, topic = yield queue.get()
                
                # Determine service rate
                service_time = random.expovariate(mu_simple if topic == 'simple' else mu_complex)
                yield self.env.timeout(service_time)
                
                waiting_time = self.env.now - arrival_time
                waiting_times.append(waiting_time)
            
# Run simulation
cc = CallCentre(env)

## Start generators
env.process(cc.call_generator('simple', LAMBDA_SIMPLE))
env.process(cc.call_generator('complex', LAMBDA_COMPLEX))

## Run sim
env.run(until=SIM_TIME)

## Print results
if len(waiting_times) > 0:
    avg_waiting_time = np.mean(waiting_times)
else:
    avg_waiting_time = 0
    
print("Average waiting time: ", avg_waiting_time)
            
            

Average waiting time:  0
